In [1]:
# General libraries
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
import joblib

# Utils functions
from utils.utils import kfold, read_datasets

In [2]:
x_train, x_test, y_train, y_test = read_datasets(
    'x_train.csv',
    'x_test.csv',
    'y_train.csv',
    'y_test.csv'
)

In [3]:
folds = kfold()

In [4]:
# Specify range of hyperparameters to tune
hyper_params = {
    'hidden_layer_sizes': [(100,), (80,), (110,)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.002, 0.005],
    'power_t': [0.5],
    'max_iter': [200],
    'shuffle': [True],
    'momentum': [0.5, 0.8, 0.9, 1],
    'early_stopping': [False]

}



# Call GridSearchCV()
model_cv = RandomizedSearchCV(
    estimator = MLPRegressor(),
    param_distributions = hyper_params,
    n_iter=200,
    scoring= 'r2',
    cv = folds,
    verbose = 2,
    return_train_score=True,
    n_jobs = -1,
    refit = True
    )


# Fit the model
best_model = model_cv.fit(x_train, np.ravel(y_train)) 

print(model_cv.best_params_)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [ 9.81603913e-02  1.00465137e-01 -1.40852897e-02  1.15497644e-01
  1.04774033e-01 -9.06570808e-01  9.75181224e-02 -3.45524627e-01
  1.01560400e-01 -8.89190819e-01 -4.06274978e-02  1.23785871e-01
  1.12835855e-02  1.09131123e-01  9.40955483e-02 -1.11426117e-03
  1.07542876e-01  9.32414129e-02 -2.01440666e+00 -6.11257238e-01
  2.19502477e-02  9.77335807e-02  1.40549049e-03 -2.87264428e-02
  7.89742114e-02 -3.90683538e-01  1.05118206e-01  9.04252641e-02
  5.65486424e-02  1.01807652e-01  1.27720346e-01  1.63908513e-01
  9.67033824e-02 -2.98345990e+00  1.08556925e-01  1.10108676e-01
  9.59120987e-02 -1.24413686e-02 -8.73196048e-01  1.04445524e-01
  9.65603990e-02  4.62394018e-02 -1.77177522e-01  5.77671200e-03
 -2.20309536e-01  1.13751833e-01 -2.81164297e-02 -5.28135168e-01
 -1.96807676e-01 -4.20895004e-01  1.41497298e-01  5.78230907e-0

{'solver': 'sgd', 'shuffle': True, 'power_t': 0.5, 'momentum': 0.8, 'max_iter': 200, 'learning_rate_init': 0.005, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (110,), 'early_stopping': False, 'alpha': 0.0001, 'activation': 'tanh'}


/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [5]:
# Create new model with best_params_ from grid search

mlp_best = best_model.best_estimator_

In [6]:
# Get the results for each split

def get_best_model_cv_split_results(best_model, n_splits=10, set_type='train'):
    results = []
    best_index = best_model.best_index_
    for i in range(0, n_splits):
        current_split = 'split{}_{}_score'.format(i, set_type)
        split_result = best_model.cv_results_[current_split][best_index]
        results.append(split_result)

    return results

print("Train Results: {}".format(get_best_model_cv_split_results(best_model, 10, 'train')))
print("Test Results: {}".format(get_best_model_cv_split_results(best_model, 10, 'test')))

Train Results: [0.28967482293825486, 0.29334147686826706, 0.2594181815892421, 0.2675555340596826, 0.2616288527477586, 0.33947715522991484, 0.271971219783791, 0.2876293371964239, 0.2896269347595166, 0.2722518188592422]
Test Results: [0.27890663144582795, 0.12863441272748244, 0.3077986840470681, 0.2685943621283644, 0.23398626825114377, -0.29036166681079223, 0.1718545711009628, 0.12232688180674234, 0.19160842064950478, 0.26154447469035813]


In [7]:
#Get the mean for the train and test

train_mean = sum(get_best_model_cv_split_results(best_model, 10, 'train'))/10
test_mean = sum(get_best_model_cv_split_results(best_model, 10, 'test'))/10

print("Train mean: {}".format(train_mean))
print("Validation mean: {}".format(test_mean))

Train mean: 0.28325753340320936
Test mean: 0.16748930400366624


In [8]:
r2 = r2_score(y_test, mlp_best.predict(x_test))
print("The r2 score on test set: {:.4f}".format(r2))

The r2 score on test set: 0.1514


# Saving trained model

In [9]:
filename = '../models/mlpRegressor_model.joblib'
joblib.dump(mlp_best, filename)

['../models/mlpRegressor.joblib']